In [1]:
import pandas as pd
import re
import csv
from Bio import SeqIO

In [2]:
AA3_TO_1 = {
    "Ala":"A","Arg":"R","Asn":"N","Asp":"D","Cys":"C",
    "Gln":"Q","Glu":"E","Gly":"G","His":"H","Ile":"I",
    "Leu":"L","Lys":"K","Met":"M","Phe":"F","Pro":"P",
    "Ser":"S","Thr":"T","Trp":"W","Tyr":"Y","Val":"V",
    "Ter":"*","Stop":"*",
}

def read_table_auto(path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    try:
        dialect = csv.Sniffer().sniff(sample, delimiters=[",", "\t", ";"])
        sep = dialect.delimiter
    except Exception:
        sep = "\t" if "\t" in sample else ","
    return pd.read_csv(path, sep=sep, engine="python", dtype=str)

def parse_hgvsp(hgvsp):
    if not isinstance(hgvsp, str) or hgvsp.strip() == "":
        return None
    h = hgvsp.replace("p.", "")
    if any(x in h for x in ["fs", "del", "ins", "dup", "ext"]):
        return None

    # p.K250N / p.S500= / p.Y107*
    m = re.fullmatch(r"([A-Z\*])(\d+)([A-Z\*\=])", h)
    if m:
        ref, pos, alt = m.group(1), int(m.group(2)), m.group(3)
        if alt == "=":
            alt = ref
        return ref, pos, alt

    # p.Lys250Asn / p.Tyr107Ter
    m = re.fullmatch(r"([A-Z][a-z]{2})(\d+)([A-Z][a-z]{2}|\*)", h)
    if m:
        ref3, pos, alt3 = m.group(1), int(m.group(2)), m.group(3)
        ref = AA3_TO_1.get(ref3)
        alt = "*" if alt3 == "*" else AA3_TO_1.get(alt3)
        return ref, pos, alt

    return None

def pick_hgvsp(row):
    for col in ["HGVSp_Short", "Protein_Change", "HGVSp"]:
        v = row.get(col)
        if isinstance(v, str) and v.strip():
            return v.strip()
    return ""


In [4]:
uniprot_fasta = "/data/RichardLuLab/Eric/MUTNELO/uniprot_human.fasta"

prot_db = {}
for rec in SeqIO.parse(uniprot_fasta, "fasta"):
      if "|" in rec.id:
        acc = rec.id.split("|")[1]
    else:
        acc = rec.id
    prot_db[acc] = str(rec.seq)

print("Loaded UniProt proteins:", len(prot_db))

Loaded UniProt proteins: 205155


In [6]:
csv_file = "/data/RichardLuLab/Eric/MUTNELO/mutantGBM.csv"
df = read_table_auto(csv_file)

print("Rows in CSV:", df.shape[0])
print("Columns:", df.columns.tolist())

Rows in CSV: 22073
Columns: ['Hugo_Symbol', 'Entrez_Gene_Id', 'NCBI_Build', 'Chromosome', 'Start_Position', 'End_Position', 'Strand', 'Consequence', 'Variant_Classification', 'Variant_Type', 'HGVSc', 'HGVSp', 'HGVSp_Short', 'Transcript_ID', 'RefSeq', 'Codons', 'Hotspot', 'Transcript_Exon', 'CGC_Other_Diseases', 'Transcript_Position', 'GO_Biological_Process', 'CGC_Tumor_Types_Somatic', 'TCGAscape_Deletion_Peaks', 'UniProt_AApos', 'OREGANNO_ID', 'SwissProt_entry_Id', 'cDNA_Change', 'Other_Transcripts', 'COSMIC_total_alterations_in_gene', 'Codon_Change', 'COSMIC_fusion_genes', 'DNARepairGenes_Role', 'SwissProt_acc_Id', 'DrugBank', 'Genome_Change', 'OREGANNO_Values', 'gc_content', 'ref_context', 'COSMIC_overlapping_mutations', 'CGC_Translocation_Partner', 'Protein_Change', 'Transcript_Strand', 'Refseq_mRNA_Id', 'Refseq_prot_Id']


In [7]:
out_fasta = "/data/RichardLuLab/Eric/MUTNELO/mutant.fasta"
skipped_tsv = "/data/RichardLuLab/Eric/MUTNELO/skipped.tsv"

out = []
skipped = []

for idx, row in df.iterrows():
    gene = row.get("Hugo_Symbol", "NA")
    acc = row.get("SwissProt_acc_Id", "")
    var_class = row.get("Variant_Classification", row.get("Consequence", "NA"))
    genome_change = row.get("Genome_Change", "")

    hgvsp = pick_hgvsp(row)
    parsed = parse_hgvsp(hgvsp)
    if parsed is None:
        skipped.append((idx, gene, acc, hgvsp, "unsupported_HGVSp"))
        continue

    ref, pos, alt = parsed

    if acc not in prot_db:
        skipped.append((idx, gene, acc, hgvsp, "protein_not_found"))
        continue

    wt = prot_db[acc]
    if pos < 1 or pos > len(wt) or wt[pos-1] != ref:
        skipped.append((idx, gene, acc, hgvsp, "aa_mismatch"))
        continue

    if alt == "*":
        mut = wt[:pos-1]   # truncate before stop
    else:
        mut = wt[:pos-1] + alt + wt[pos:]

    header = f">{gene}|{hgvsp}|{acc}"
    if genome_change:
        header += f"|{genome_change}"

    out.append(header)
    for i in range(0, len(mut), 60):
        out.append(mut[i:i+60])

# write files
with open(out_fasta, "w") as f:
    f.write("\n".join(out))

pd.DataFrame(
    skipped,
    columns=["row","Gene","UniProt_ACC","HGVSp","Reason"]
).to_csv(skipped_tsv, sep="\t", index=False)

print("Mutant FASTA records:", sum(1 for x in out if x.startswith(">")))
print("Skipped variants:", len(skipped))


Mutant FASTA records: 17799
Skipped variants: 4274


In [10]:
!head -n 6 /data/RichardLuLab/Eric/MUTNELO/mutant.fasta
!cut -f5 /data/RichardLuLab/Eric/MUTNELO/skipped.tsv | sort | uniq -c | sort -nr


>SLC45A1|p.S500=|Q9Y2W3|g.chr1:8395553C>T
MIPAASSTPPGDALFPSVAPQDFWRSQVTGYSGSVTRHLSHRANNFKRHPKRRKCIRPSP
PPPPNTPCPLELVDFGDLHPQRSFRELLFNGCILFGIEFSYAMETAYVTPVLLQMGLPDQ
LYSLVWFISPILGFLLQPLLGAWSDRCTSRFGRRRPFILVLAIGALLGLSLLLNGRDIGI
ALADVTGNHKWGLLLTVCGVVLMDFSADSADNPSHAYMMDVCSPADQDRGLNIHALLAGL
GGGFGYVVGGIHWDKTGFGRALGGQLRVIYLFTAVTLSVTTVLTLVSIPERPLRPPSEKR
   2115 aa_mismatch
   1389 unsupported_HGVSp
    770 protein_not_found
      1 Reason
